In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

**1. data cleaning**

In [ ]:
#import us data 
us_df=pd.read_csv('../input/youtube-new/USvideos.csv')

In [ ]:
us_df.shape

In [ ]:
us_df.nunique()
#as we can see, a video can occur several times (continuous popularity)

In [ ]:
us_df['comments_disabled'].value_counts()

In [ ]:
us_df.isnull().sum(axis=1).sum()

In [ ]:
#drop all the NA values
us_df1=us_df.dropna()

In [ ]:
us_df1.nunique()

In [ ]:
us_df1['like_rate']=us_df1['likes']/(us_df1['likes']+us_df1['dislikes'])

In [ ]:
us_df1['publish_time'] = pd.to_datetime(us_df1['publish_time'], format='%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
us_df1['p_year']=us_df1['publish_time'].dt.year
us_df1['p_month'] = us_df1['publish_time'].dt.month
us_df1['p_day'] = us_df1['publish_time'].dt.day
us_df1["p_weekday"]=us_df1['publish_time'].dt.weekday
us_df1['p_time'] = us_df1['publish_time'].dt.time

In [ ]:
us_df1.loc[(us_df1["category_id"] == 1),"category_name"] = 'Film and Animation'
us_df1.loc[(us_df1["category_id"] == 2),"category_name"] = 'Autos and Vehicles'
us_df1.loc[(us_df1["category_id"] == 10),"category_name"] = 'Music'
us_df1.loc[(us_df1["category_id"] == 15),"category_name"] = 'Pets and Animals'
us_df1.loc[(us_df1["category_id"] == 17),"category_name"] = 'Sports'
us_df1.loc[(us_df1["category_id"] == 18),"category_name"] = 'Short Movies'
us_df1.loc[(us_df1["category_id"] == 19),"category_name"] = 'Travel and Events'
us_df1.loc[(us_df1["category_id"] == 20),"category_name"] = 'Gaming'
us_df1.loc[(us_df1["category_id"] == 21),"category_name"] = 'Videoblogging'
us_df1.loc[(us_df1["category_id"] == 22),"category_name"] = 'People and Blogs'
us_df1.loc[(us_df1["category_id"] == 23),"category_name"] = 'Comedy'
us_df1.loc[(us_df1["category_id"] == 24),"category_name"] = 'Entertainment'
us_df1.loc[(us_df1["category_id"] == 25),"category_name"] = 'News and Politics'
us_df1.loc[(us_df1["category_id"] == 26),"category_name"] = 'How to and Style'
us_df1.loc[(us_df1["category_id"] == 27),"category_name"] = 'Education'
us_df1.loc[(us_df1["category_id"] == 28),"category_name"] = 'Science and Technology'
us_df1.loc[(us_df1["category_id"] == 29),"category_name"] = 'Non Profits and Activism'
us_df1.loc[(us_df1["category_id"] == 30),"category_name"] = 'Movies'
us_df1.loc[(us_df1["category_id"] == 31),"category_name"] = 'Anime/Animation'
us_df1.loc[(us_df1["category_id"] == 32),"category_name"] = 'Action/Adventure'
us_df1.loc[(us_df1["category_id"] == 33),"category_name"] = "Classics"
us_df1.loc[(us_df1["category_id"] == 34),"category_name"] = 'Comedy'
us_df1.loc[(us_df1["category_id"] == 35),"category_name"] = 'Documentary'
us_df1.loc[(us_df1["category_id"] == 36),"category_name"] = 'Drama'
us_df1.loc[(us_df1["category_id"] == 37),"category_name"] = 'Family'
us_df1.loc[(us_df1["category_id"] == 38),"category_name"] = 'Foreign'
us_df1.loc[(us_df1["category_id"] == 39),"category_name"] = 'Horror'
us_df1.loc[(us_df1["category_id"] == 40),"category_name"] = "Sci-Fi/Fantasy"
us_df1.loc[(us_df1["category_id"] == 41),"category_name"] = "Thriller"
us_df1.loc[(us_df1["category_id"] == 42),"category_name"] = "Shorts"
us_df1.loc[(us_df1["category_id"] == 43),"category_name"] = "Shows"
us_df1.loc[(us_df1["category_id"] == 44),"category_name"] = "Trailers"

In [ ]:
us_df2=us_df1.copy()

In [ ]:
len(us_df2.columns)

In [ ]:
us_df2.columns

In [ ]:
#reorder for convenience
new_order=['video_id',  'title', 'channel_title', 'category_id','category_name',
        'tags','thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description', 'trending_date','publish_time', 'p_year', 'p_month',
       'p_weekday', 'p_time', 'views', 'likes', 'dislikes','like_rate', 'comment_count']
us_df3=us_df2.reindex(columns=new_order)

In [ ]:
us_df3["video_id"].value_counts()

In [ ]:
us_df3["appearances"]=us_df3.groupby("video_id")['video_id'].transform('size')

In [ ]:
us_df3.columns

In [ ]:
con=us_df3['video_id']=='2kyS6SvSYSE'
us_df3[con]

In [ ]:
#group by video id and order by trending date
us_df4=us_df3.groupby('video_id').apply(lambda x: x.sort_values('views'))

In [ ]:
us_df4[["trending_date","publish_time","views","appearances"]].head(10)

In [ ]:
#only keep the latest trending one
us_df5=us_df4.drop_duplicates(['video_id'],keep='last')

In [ ]:
us_df5 = us_df5.rename(columns={"trending_date": "last_trending"})

In [ ]:
us_df5.head()

In [ ]:
#writer = pd.ExcelWriter('us_youtube_cleaned.xlsx', engine='xlsxwriter')
#us_df5.to_excel(writer, sheet_name='Sheet1')
#writer.save()

In [ ]:
#transform false to 0, true to 1 for boolean columns
us_df5[['comments_disabled', 'ratings_disabled','video_error_or_removed']]=us_df5[['comments_disabled', 'ratings_disabled','video_error_or_removed']].astype('int')

**2. exploratory data analysis**

In [ ]:
#2.1 duration analysis (trending days)
import matplotlib.pyplot as plt
plt.hist(us_df5['appearances'], bins=30, normed=True, alpha=0.6,
         histtype='stepfilled', color='green',range=[0,30],
         edgecolor='none')
plt.title("appearances distribution in US")
plt.savefig('p1.png')

In [ ]:
#2.2 views/likes distribution;
#    views v.s. likes and dislikes (by cat?);
#    views v.s. like rate;
#    views by weekday (just as an example)
import matplotlib.pyplot as plt
plt.hist(np.log(us_df5['views']), bins=30, normed=True, alpha=0.6,
         histtype='stepfilled', color='red',range=[0,20],
         edgecolor='none')
plt.title("log.views distribution in US")
plt.savefig('p2.png')

highly right skewed

In [ ]:
import matplotlib.pyplot as plt
plt.hist(np.log(us_df5['likes']), bins=30, normed=True, alpha=0.6,
         histtype='stepfilled', color='blue',range=[0,20],
         edgecolor='none')
plt.title("log.likes distribution in US")
plt.savefig('p3.png')

In [ ]:
plt.plot(np.log(us_df5['likes']), np.log(us_df5['views']), 'o', color='orange',alpha=0.4)
plt.xlabel("log.likes")
plt.ylabel("log.views")
plt.title("log.views v.s. log.likes in US")
plt.savefig('p4.png')

In [ ]:
#2.2 heatmap between variables (just an example)
def heatmap(x, y, size):
    fig, ax = plt.subplots()
    
    # Mapping from column names to integer coordinates
    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    size_scale = 1000
    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=size * size_scale, # Vector of square sizes, proportional to size parameter
        marker='s' # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)
       
data = us_df5
columns = ['views', 'p_weekday', 'comments_disabled', 'ratings_disabled', 'like_rate','comment_count','appearances'] 
corr = data[columns].corr()
corr = pd.melt(corr.reset_index(), id_vars='index') 
corr.columns = ['x', 'y', 'value']
heatmap(
    x=corr['x'],
    y=corr['y'],
    size=corr['value'].abs()
)
plt.savefig('p5.png')

In [ ]:
#2.3 time series analysis

In [ ]:
#2.4 category

In [ ]:
#WIP: when we are done with the models, there will be more of this!

#this code takes too long to run
plt.bar(us_df3["category_name"], us_df3["views"], align='center', alpha=0.5)
plt.ylabel('category')
plt.ylabel('views')
plt.title('views by category in US')
plt.show()

**3. modeling**

def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
us_df5['tokenized_column'] = us_df5.title.apply(custom_tokenize)

tokenized_word = us_df5['tokenized_column']

master_list = []
for i in tokenized_word:
    master_list.append(i)


flat_list = []
for sublist in master_list:
    for item in sublist:
        flat_list.append(item)


def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

print(listToString(flat_list))


tokenized_word = listToString(flat_list)


from nltk.probability import FreqDist
fdist = FreqDist(tokenized_word)
print(fdist)


fdist.most_common(10)

import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()

import re 
from nltk import sent_tokenize
dataset = nltk.sent_tokenize(tokenized_word) 
for i in range(len(dataset)): 
    dataset[i] = dataset[i].lower() 
    dataset[i] = re.sub(r'\W', ' ', dataset[i]) 
    dataset[i] = re.sub(r'\s+', ' ', dataset[i])

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')

[word for word in dataset if word not in stop_words]

word2count = {} 
for data in dataset: 
    words = nltk.word_tokenize(data) 
    for word in words: 
        if word not in word2count.keys(): 
            word2count[word] = 1
        else: 
            word2count[word] += 1

import heapq 
freq_words = heapq.nlargest(100, word2count, key=word2count.get)

X = [] 
for data in dataset: 
    vector = [] 
    for word in freq_words: 
        if word in nltk.word_tokenize(data): 
            vector.append(1) 
        else: 
            vector.append(0) 
    X.append(vector) 
X = np.asarray(X)

from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = us_df5.title
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

In [ ]:
from nltk.stem import PorterStemmer
stop = stopwords.words('english')

In [ ]:
ps = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents
text = us_df5.title
text = text.str.replace('\d+', '')
text = text.apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
text = text.str.lower().map(ps.stem)
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vector.toarray())

In [ ]:
vector.toarray().tolist()